In [3]:
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit.quantum_info import SparsePauliOp, Pauli
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.units import DistanceUnit
import numpy as np
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.circuit import QuantumCircuit
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.quantum_info import SparsePauliOp
from qiskit.synthesis.evolution import SuzukiTrotter
from qiskit import transpile
from qiskit.transpiler import CouplingMap
from qiskit.transpiler import CouplingMap

n_qubits = 4
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.742",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

molecule = driver.run()
mapper = JordanWignerMapper()
cost_h = mapper.map(molecule.hamiltonian.second_q_op())

driver_scaling = 1.0
driver_h = SparsePauliOp.from_list([
    ("XIII", -1.0*driver_scaling),
    ("IXII", -1.0*driver_scaling),
    ("IIXI", -1.0*driver_scaling),
    ("IIIX", -1.0*driver_scaling),
])


def build_commutator(op_a: SparsePauliOp, op_b: SparsePauliOp) -> SparsePauliOp:
    ab = op_a @ op_b
    ba = op_b @ op_a
    comm_pre= ab - ba
    comm = comm_pre*1j
    return comm.simplify()

comm_h=build_commutator(driver_h, cost_h)

def falqon_layer(qc,cost_h,driver_h,beta_k,delta_t):
    #synth = SuzukiTrotter(reps=1)   ,synthesis=synth
    U_c= PauliEvolutionGate(cost_h,delta_t)
    U_d= PauliEvolutionGate(driver_h,delta_t*beta_k)
    qc.append(U_c, range(qc.num_qubits))
    qc.append(U_d, range(qc.num_qubits))



def main_loop(qc,cost_h,driver_h,comm_h,delta_t,beta_0,n_steps):
    beta =[beta_0]
    energies=[]
    state = Statevector.from_label("+" * n_qubits)
    for i in range(n_steps):
        qc_layer = QuantumCircuit(n_qubits)
        falqon_layer(qc_layer,cost_h,driver_h,beta[i],delta_t)
        qc.compose(qc_layer, inplace=True)
        state = state.evolve(qc_layer) 
        energy = state.expectation_value(cost_h).real
        energies.append(energy)
        next_beta =-1*state.expectation_value(comm_h).real
        beta.append(next_beta)
        print(f"第{i + 1}步：能量 = {energy:.8f} Ha | β_{i} = {beta[i]:+.6f}")


        
    return beta, energies, qc


def uniform_superposition_circuit(n_qubits=4):
    qc = QuantumCircuit(n_qubits)
    qc.h(range(n_qubits))  # 对所有 qubit 施加 H 门
    return qc

qc_initial= uniform_superposition_circuit()

n_steps = 200
beta_0 = 0.01
delta_t = 0.03
s_beta, s_energies, final_circuit = main_loop(
    qc=qc_initial,
    cost_h=cost_h,
    driver_h=driver_h,
    comm_h=comm_h,
    delta_t=delta_t,
    beta_0=beta_0,
    n_steps=n_steps
)

第1步：能量 = -0.76746786 Ha | β_0 = +0.010000
第2步：能量 = -0.76759209 Ha | β_1 = +0.049917
第3步：能量 = -0.76801466 Ha | β_2 = +0.099781
第4步：能量 = -0.76888278 Ha | β_3 = +0.149519
第5步：能量 = -0.77034062 Ha | β_4 = +0.199023
第6步：能量 = -0.77252716 Ha | β_5 = +0.248144
第7步：能量 = -0.77557328 Ha | β_6 = +0.296681
第8步：能量 = -0.77959804 Ha | β_7 = +0.344375
第9步：能量 = -0.78470411 Ha | β_8 = +0.390903
第10步：能量 = -0.79097267 Ha | β_9 = +0.435884
第11步：能量 = -0.79845798 Ha | β_10 = +0.478877
第12步：能量 = -0.80718221 Ha | β_11 = +0.519399
第13步：能量 = -0.81713128 Ha | β_12 = +0.556939
第14步：能量 = -0.82825219 Ha | β_13 = +0.590987
第15步：能量 = -0.84045283 Ha | β_14 = +0.621066
第16步：能量 = -0.85360453 Ha | β_15 = +0.646763
第17步：能量 = -0.86754749 Ha | β_16 = +0.667763
第18步：能量 = -0.88209898 Ha | β_17 = +0.683878
第19步：能量 = -0.89706327 Ha | β_18 = +0.695066
第20步：能量 = -0.91224250 Ha | β_19 = +0.701439
第21步：能量 = -0.92744705 Ha | β_20 = +0.703254
第22步：能量 = -0.94250454 Ha | β_21 = +0.700897
第23步：能量 = -0.95726647 Ha | β_22 = +0.694857
第24步：能量

In [4]:
driver_h = SparsePauliOp.from_list([
    ("XIII", -1.0),
    ("IXII", -1.0),
    ("IIXI", -1.0),
    ("IIIX", -1.0),
])

cost_h = SparsePauliOp.from_list([
    ("ZIII", -1.0),
    ("IZII", -1.0),
    ("IIZI", -1.0),
    ("IIIZ", -1.0),
])


def build_commutator(op_a: SparsePauliOp, op_b: SparsePauliOp) -> SparsePauliOp:
    ab = op_a @ op_b
    ba = op_b @ op_a
    comm_pre= ab - ba
    comm = comm_pre*1j
    return comm.simplify()

comm_h=build_commutator(driver_h, cost_h)

def falqon_layer(qc,cost_h,driver_h,beta_k,delta_t):
    #synth = SuzukiTrotter(reps=1)   ,synthesis=synth
    U_c= PauliEvolutionGate(cost_h,delta_t)
    U_d= PauliEvolutionGate(beta_k*driver_h,delta_t)
    qc.append(U_c, range(qc.num_qubits))
    qc.append(U_d, range(qc.num_qubits))



def main_loop(qc,cost_h,driver_h,comm_h,delta_t,beta_0,n_steps):
    beta =[beta_0]
    energies=[]
    state = Statevector.from_label("+" * n_qubits)
    for i in range(n_steps):
        qc_layer = QuantumCircuit(n_qubits)
        falqon_layer(qc_layer,cost_h,driver_h,beta[i],delta_t)
        qc.compose(qc_layer, inplace=True)
        state = state.evolve(qc_layer) 
        energy = state.expectation_value(cost_h).real
        energies.append(energy)
        next_beta =-1*state.expectation_value(comm_h).real
        beta.append(next_beta)
        print(f"第{i + 1}步：能量 = {energy:.8f} | β_{i} = {beta[i]:+.6f}")


        
    return beta, energies, qc


def uniform_superposition_circuit(n_qubits=4):
    qc = QuantumCircuit(n_qubits)
    qc.h(range(n_qubits))  # 对所有 qubit 施加 H 门
    return qc

qc_initial= uniform_superposition_circuit()

n_steps = 2000
beta_0 = 0.01
delta_t = 0.03
s_beta, s_energies, final_circuit = main_loop(
    qc=qc_initial,
    cost_h=cost_h,
    driver_h=driver_h,
    comm_h=comm_h,
    delta_t=delta_t,
    beta_0=beta_0,
    n_steps=n_steps
)


第1步：能量 = -0.00014391 | β_0 = +0.010000
第2步：能量 = -0.01392452 | β_1 = +0.479712
第3步：能量 = -0.05499943 | β_2 = +0.957293
第4步：能量 = -0.13596994 | β_3 = +1.427872
第5步：能量 = -0.26706029 | β_4 = +1.880895
第6步：能量 = -0.45365866 | β_5 = +2.297959
第7步：能量 = -0.69336039 | β_6 = +2.652673
第8步：能量 = -0.97400038 | β_7 = +2.914244
第9步：能量 = -1.27470463 | β_8 = +3.055653
第10步：能量 = -1.57086694 | β_9 = +3.064395
第11步：能量 = -1.84117275 | β_10 = +2.950123
第12步：能量 = -2.07283899 | β_11 = +2.743428
第13步：能量 = -2.26255273 | β_12 = +2.485581
第14步：能量 = -2.41385288 | β_13 = +2.215594
第15步：能量 = -2.53349177 | β_14 = +1.961649
第16步：能量 = -2.62867569 | β_15 = +1.739132
第17步：能量 = -2.70565703 | β_16 = +1.553126
第18步：能量 = -2.76931975 | β_17 = +1.402258
第19步：能量 = -2.82326664 | β_18 = +1.281990
第20步：能量 = -2.87007585 | β_19 = +1.186799
第21步：能量 = -2.91156622 | β_20 = +1.111371
第22步：能量 = -2.94901540 | β_21 = +1.051120
第23步：能量 = -2.98332280 | β_22 = +1.002335
第24步：能量 = -3.01512573 | β_23 = +0.962132
第25步：能量 = -3.04488043 | β_24 = +0.9